### Exploratory Data Analysis of "Arctic Rivers" data
Modeled stream temp and stream flow data. The values are mapped to [MERIT HYDRO](https://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/) unique stream IDs.

In [29]:
import xarray as xr
from pathlib import Path

In [30]:
dir = Path('/beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/')
# get files from model hC2LE7 in year 1990 only, plus "historical" from 1990
fps = list(dir.glob('1990*hC2LE7*.nc'))
hist = list(dir.glob('1990*historical*.nc'))

fps += hist

In [31]:
fps

[PosixPath('/beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/1990_hC2LE7_WT.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/1990_hC2LE7_Q.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/1990_hC2LE7_climate.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/1990_historical_Q.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/1990_historical_climate.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/Arctic_Rivers_Data/1990_historical_WT.nc')]

In [32]:
# separate modeled from "historical"
wt_modeled = xr.open_dataset(fps[0])
q_modeled = xr.open_dataset(fps[1])
clim_modeled = xr.open_dataset(fps[2])

q_hist = xr.open_dataset(fps[3])
clim_hist = xr.open_dataset(fps[4])
wt_hist = xr.open_dataset(fps[5])


In [33]:
# water temperature
# "no_seg" is probably "number of segments"
# "hru" is probably "hydrological response unit" - this is the unique ID of the MERIT HYDRO river segment, not an actual float value

wt_modeled

<xarray.Dataset> Size: 600MB
Dimensions:      (time: 365, no_seg: 2, hru: 34208)
Coordinates:
  * time         (time) datetime64[ns] 3kB 1990-01-01 1990-01-02 ... 1990-12-31
  * no_seg       (no_seg) int32 8B 1 2
  * hru          (hru) float64 274kB 8.1e+07 8.1e+07 ... 8.103e+07 8.103e+07
Data variables:
    T_stream     (time, no_seg, hru) float64 200MB ...
    T_air        (time, no_seg, hru) float64 200MB ...
    T_headwater  (time, no_seg, hru) float64 200MB ...

In [34]:
wt_hist

<xarray.Dataset> Size: 600MB
Dimensions:      (time: 365, no_seg: 2, hru: 34208)
Coordinates:
  * hru          (hru) float64 274kB 8.103e+07 8.103e+07 ... 8.103e+07 8.103e+07
  * no_seg       (no_seg) int32 8B 1 2
  * time         (time) datetime64[ns] 3kB 1990-01-01 1990-01-02 ... 1990-12-31
Data variables:
    T_air        (time, no_seg, hru) float64 200MB ...
    T_headwater  (time, no_seg, hru) float64 200MB ...
    T_stream     (time, no_seg, hru) float64 200MB ...
Attributes:
    history:  Sat May 18 08:04:24 2024: ncks --mk_rec_dmn time /glade/derecho...
    NCO:      netCDF Operators version 5.1.9 (Homepage = http://nco.sf.net, C...

In [59]:
# compare modeled vs "historical" for a water temperature from a certain date, segment, and hru
print(wt_modeled['T_stream'].sel(time='1990-01-01', no_seg=1, hru=81000076).values)
print(wt_hist['T_stream'].sel(time='1990-01-01', no_seg=1, hru=81000076).values)

1.84
3.74


In [35]:
# streamflow ("Q" is discharge in m^3/s), here called "IRFroutedRunoff"
# for some reason, "seg" refers to the unique ID of the MERIT HYDRO river segment, instead of "hru" used in the water temp file

q_modeled

<xarray.Dataset> Size: 50MB
Dimensions:          (time: 365, seg: 34346)
Coordinates:
  * time             (time) datetime64[ns] 3kB 1990-01-01T12:00:00 ... 1990-1...
  * seg              (seg) int64 275kB 81003498 81003523 ... 81034852 81034890
Data variables:
    IRFroutedRunoff  (time, seg) float32 50MB ...
Attributes:
    version:  v1.2.1

In [36]:
q_hist

<xarray.Dataset> Size: 53MB
Dimensions:          (time: 365, seg: 35923)
Coordinates:
  * time             (time) datetime64[ns] 3kB 1990-01-01T12:00:00 ... 1990-1...
  * seg              (seg) int64 287kB 81003498 81003523 ... 81034852 81034890
Data variables:
    IRFroutedRunoff  (time, seg) float32 52MB ...
Attributes:
    version:  v1.2.1

In [61]:
# compare modeled vs "historical" for streamflow from a certain date, segment, and hru
print(q_modeled['IRFroutedRunoff'].sel(time='1990-01-01', seg=81000076).values)
print(q_hist['IRFroutedRunoff'].sel(time='1990-01-01', seg=81000076).values)

[0.03519676]
[0.02681552]


In [37]:
# climate data (precipitation, temperature, etc.)
# gridded with XY coords, not river segment IDs
# these are the inputs used to generate the WT and Q modeled data, maybe not useful in this context
clim_modeled

<xarray.Dataset> Size: 4GB
Dimensions:        (time: 365, hist_interval: 2, lon: 782, lat: 662)
Coordinates:
  * time           (time) object 3kB 1990-01-02 00:00:00 ... 1991-01-01 00:00:00
  * lon            (lon) float32 3kB 184.2 184.3 184.3 ... 225.7 225.7 225.8
  * lat            (lat) float32 3kB 50.01 50.04 50.07 ... 69.83 69.86 69.89
Dimensions without coordinates: hist_interval
Data variables:
    time_bounds    (time, hist_interval) object 6kB ...
    H2OSNO         (time, lat, lon) float32 756MB ...
    QFLX_EVAP_TOT  (time, lat, lon) float32 756MB ...
    RAIN           (time, lat, lon) float32 756MB ...
    SNOW           (time, lat, lon) float32 756MB ...
    TSA            (time, lat, lon) float32 756MB ...
Attributes: (12/39)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   Sat May 18 10:11:35 2024: ncrc...
    source:                                    Community Terrestrial Systems ...
    hostname:                                  derecho
    ...                                        ...
    ctype_urban_impervious_road:               74
    ctype_urban_pervious_road:                 75
    cft_c3_crop:                               1
    cft_c3_irrigated:                          2
    time_period_freq:                          day_1
    NCO:                                       netCDF Operators version 5.1.9...

In [38]:
clim_hist

<xarray.Dataset> Size: 4GB
Dimensions:        (time: 365, hist_interval: 2, lon: 782, lat: 662)
Coordinates:
  * time           (time) datetime64[ns] 3kB 1990-01-02 ... 1991-01-01
  * lon            (lon) float32 3kB 184.2 184.3 184.3 ... 225.7 225.7 225.8
  * lat            (lat) float32 3kB 50.01 50.04 50.07 ... 69.83 69.86 69.89
Dimensions without coordinates: hist_interval
Data variables:
    time_bounds    (time, hist_interval) datetime64[ns] 6kB ...
    H2OSNO         (time, lat, lon) float32 756MB ...
    QFLX_EVAP_TOT  (time, lat, lon) float32 756MB ...
    RAIN           (time, lat, lon) float32 756MB ...
    SNOW           (time, lat, lon) float32 756MB ...
    TSA            (time, lat, lon) float32 756MB ...
Attributes: (12/39)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   Sat May 18 12:49:37 2024: ncrc...
    source:                                    Community Terrestrial Systems ...
    hostname:                                  cheyenne
    ...                                        ...
    ctype_urban_impervious_road:               74
    ctype_urban_pervious_road:                 75
    cft_c3_crop:                               1
    cft_c3_irrigated:                          2
    time_period_freq:                          day_1
    NCO:                                       netCDF Operators version 5.1.9...